In [15]:
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary

from moduleZoo.graphs import GraphConv

In [16]:
class TargetModel(nn.Module):
    def __init__(self, db = False) -> None:
        super().__init__()
        self.module = GraphConv(128, 256, k=10, dynamic_batching=db)

    def enable_dynamic_batching(self) -> None:
        self.module.db = True

    def forward(self, x:torch.Tensor, n_nodes = None) -> torch.Tensor:
        return self.module(x, n_nodes)

In [17]:
module = TargetModel(False)

In [18]:
module = module.to('cuda')

In [19]:
data = torch.rand((10, 100, 128), device='cuda')
print(f'{data.shape = }')
# n_nodes = [1]*(256//2) + [2]*(256//4)
# n_nodes = np.array(n_nodes*2)

data.shape = torch.Size([10, 100, 128])


In [20]:
result1 = module(data)

Using static forward
1: x.shape = torch.Size([10, 100, 128])
2: x.shape = torch.Size([10, 100, 10, 256])


In [21]:
module.module.db = True

In [22]:
data = data.view((10*100, 128))
n_nodes = np.ones((10, ), dtype=np.int16)*100

In [23]:
result2 = module(data, n_nodes).view((10, 100, 256))

Using static dynamic batching.
1: x.shape = torch.Size([10, 100, 128])
2: x.shape = torch.Size([10, 100, 10, 256])


In [26]:
module.module.db = False

In [27]:
result3 = module(data, n_nodes).view((10, 100, 256))

Using complete dynamic batching


In [29]:
torch.all(result3 == result2)

tensor(True, device='cuda:0')

In [7]:
# torch.all((result1.detach().cpu() - result2.detach().cpu()).abs() < 1e-6)
result1.shape

torch.Size([10, 100, 256])

In [9]:
summary(module1, [500, 128])

Layer (type:depth-idx)                   Output Shape              Param #
├─GraphConv1d: 1-1                       [-1, 500, 256]            --
|    └─LinearNormActivation: 2-1         [-1, 500, 10, 256]        --
|    |    └─Linear: 3-1                  [-1, 500, 10, 256]        65,792
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.24
Forward/backward pass size (MB): 9.77
Params size (MB): 0.25
Estimated Total Size (MB): 10.26


Layer (type:depth-idx)                   Output Shape              Param #
├─GraphConv1d: 1-1                       [-1, 500, 256]            --
|    └─LinearNormActivation: 2-1         [-1, 500, 10, 256]        --
|    |    └─Linear: 3-1                  [-1, 500, 10, 256]        65,792
Total params: 65,792
Trainable params: 65,792
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.24
Forward/backward pass size (MB): 9.77
Params size (MB): 0.25
Estimated Total Size (MB): 10.26

In [11]:
_[1]

tensor([[ 54,  54,  53, 111,  55,  61,  57, 117,  83, 125,  58,  68,  66, 117,
          38,  62, 123,  59,  34,  39,  45,  65,  18, 124,  82, 112,  33,  87,
          86,  91,  38,  90,  46,  89,  11, 127,  79,   5,  20,  58,  28, 110,
          46,  81, 100,  60, 102, 116,  13,  84,  43, 124,  84,  30,  79,  40,
          66,  40,  16,  35,  88,  91,  44,  24,  74,  46,   6, 102,  93,  64,
          35,  60, 104, 117,  64,   1,  51,  46,  70,  45, 118,  17, 126,  78,
           5, 103,  64,  78, 127, 116,  60,  51,  71,  54,  86,  58, 101, 116,
          16,  59,  68, 125,  32,  62,  24,  81,  35,  91,  23,  66,  65,  89,
          55,  97,  62,  33,  79,  99,  25, 122, 123,  53, 108,  13,  13,  70,
         106,  34,  89, 105,   9,   2,  56,  52,  50,  35,  82,  88, 120,  94,
          21,  21,   7,  62,  34,  78,  22, 111, 126, 123,  47,  60,  79,  72,
          52, 103,  27,  45,  52,  24,  74,   4,  41,   4,  65,  32,   2, 109,
          24,  86,  85,  77,  20,  28,  21,  64,  30